# 🗺️ Preprocessing Absensi - Deteksi Anomali Lokasi
**Pipeline:** Raw Data → Time Transform → Coordinate Transform → DBSCAN → Centroid Kantor → Haversine → Validation → ST-DBSCAN → Anomaly Scoring → Risk Level

---

## 📦 Install Dependencies

In [ ]:
!pip install pandas numpy scikit-learn openpyxl haversine folium streamlit pydeck plotly streamlit-folium -q

## 📥 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from haversine import haversine, Unit
import warnings
warnings.filterwarnings('ignore')

print('✅ Libraries loaded')

## 📂 Load Dataset

In [ ]:
# ============================================================
# GANTI PATH SESUAI FILE ANDA
# Jika di Google Colab, upload file dulu atau mount Google Drive
# ============================================================

# Option 1: Upload langsung di Colab
# from google.colab import files
# uploaded = files.upload()
# FILE_PATH = list(uploaded.keys())[0]

# Option 2: Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# FILE_PATH = '/content/drive/MyDrive/dataset_absensi_final2.xlsx'

# Option 3: Path lokal
FILE_PATH = 'dataset_absensi_final2.xlsx'

df = pd.read_excel(FILE_PATH)
print(f'✅ Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns')
print(f'Columns: {df.columns.tolist()}')
df.head()

## 🔍 Exploratory Data Analysis

In [ ]:
print('=== INFO ===')
df.info()
print('\n=== MISSING VALUES ===')
print(df.isnull().sum())
print('\n=== JENIS ABSENSI ===')
print(df['jenis'].value_counts())
print('\n=== STATUS LOKASI ===')
print(df['status_lokasi'].value_counts())
print('\n=== JUMLAH SKPD ===')
print(f"Unique SKPD: {df['id_skpd'].nunique()}")
print(f"Unique Karyawan: {df['karyawan_id'].nunique()}")

---
## 🟡 STEP 1: Time Feature Transformation

In [ ]:
def transform_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Membuat feature waktu dari kolom tanggal_kirim:
    - timestamp_num : detik sejak epoch
    - jam            : jam (0-23)
    - menit          : menit (0-59)
    - jam_desimal    : jam + menit/60
    - weekday        : hari dalam minggu (0=Senin, 6=Minggu)
    """
    df = df.copy()
    
    # Pastikan datetime
    df['tanggal_kirim'] = pd.to_datetime(df['tanggal_kirim'])
    
    # Timestamp numerik (detik sejak epoch)
    df['timestamp_num'] = df['tanggal_kirim'].astype(np.int64) // 10**9
    
    # Ekstraksi komponen waktu
    df['jam']         = df['tanggal_kirim'].dt.hour
    df['menit']       = df['tanggal_kirim'].dt.minute
    df['jam_desimal'] = df['jam'] + df['menit'] / 60.0
    df['weekday']     = df['tanggal_kirim'].dt.weekday  # 0=Senin
    df['tanggal']     = df['tanggal_kirim'].dt.date
    
    print('✅ Time features created: timestamp_num, jam, menit, jam_desimal, weekday, tanggal')
    return df

df = transform_time_features(df)
df[['tanggal_kirim','timestamp_num','jam','menit','jam_desimal','weekday']].head()

---
## 🟠 STEP 2: Coordinate Transformation (Radian)

In [ ]:
def transform_coordinates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Konversi lat/long ke radian untuk DBSCAN metric haversine.
    Juga bersihkan koordinat yang tidak valid.
    """
    df = df.copy()
    
    # Drop baris dengan koordinat null
    before = len(df)
    df = df.dropna(subset=['lat', 'long'])
    after = len(df)
    if before != after:
        print(f'⚠️  Dropped {before - after} rows with null coordinates')
    
    # Filter koordinat tidak valid (di luar range Indonesia)
    # Indonesia: lat -11 s/d 6, long 95 s/d 141
    mask_valid = (
        (df['lat'].between(-11, 6)) &
        (df['long'].between(95, 141))
    )
    invalid_count = (~mask_valid).sum()
    if invalid_count > 0:
        print(f'⚠️  Found {invalid_count} rows with coordinates outside Indonesia range')
        df['coord_invalid'] = (~mask_valid).astype(int)
    else:
        df['coord_invalid'] = 0
    
    # Konversi ke radian
    df['lat_rad']  = np.radians(df['lat'])
    df['long_rad'] = np.radians(df['long'])
    
    print('✅ Coordinate features created: lat_rad, long_rad, coord_invalid')
    return df

df = transform_coordinates(df)
df[['lat','long','lat_rad','long_rad']].head()

---
## 🔵 STEP 3: DBSCAN Spatial Clustering

In [ ]:
def run_dbscan_spatial(
    df: pd.DataFrame,
    jenis: str,
    eps_km: float = 0.1,
    min_samples: int = 3
) -> pd.Series:
    """
    Jalankan DBSCAN spatial per SKPD untuk jenis absensi tertentu (M/P).
    
    Parameters:
    -----------
    eps_km      : radius cluster dalam km (default 100m)
    min_samples : minimum titik untuk membentuk cluster
    
    Returns:
    --------
    Series dengan label cluster per baris
    """
    # Radius bumi dalam km
    EARTH_RADIUS_KM = 6371.0
    eps_rad = eps_km / EARTH_RADIUS_KM
    
    result = pd.Series(-1, index=df.index, dtype=int)
    
    subset = df[df['jenis'] == jenis].copy()
    
    for skpd_id, group in subset.groupby('id_skpd'):
        if len(group) < min_samples:
            continue
        
        coords = group[['lat_rad', 'long_rad']].values
        
        db = DBSCAN(
            eps=eps_rad,
            min_samples=min_samples,
            algorithm='ball_tree',
            metric='haversine'
        ).fit(coords)
        
        # Offset label agar tidak overlap antar SKPD
        labels = db.labels_
        max_label = result.max() + 1
        labels_offset = np.where(labels >= 0, labels + max_label, -1)
        
        result.loc[group.index] = labels_offset
    
    return result


# ============================================================
# Konfigurasi DBSCAN - sesuaikan dengan kebutuhan
# ============================================================
EPS_KM      = 0.1   # 100 meter radius
MIN_SAMPLES = 3     # minimal 3 titik

print('🔄 Running DBSCAN for MASUK...')
df['cluster_masuk'] = run_dbscan_spatial(df, 'M', EPS_KM, MIN_SAMPLES)

print('🔄 Running DBSCAN for PULANG...')
df['cluster_pulang'] = run_dbscan_spatial(df, 'P', EPS_KM, MIN_SAMPLES)

# Noise flags
df['is_noise_masuk']  = (df['cluster_masuk']  == -1).astype(int)
df['is_noise_pulang'] = (df['cluster_pulang'] == -1).astype(int)

# Cluster size
cluster_size_masuk  = df[df['jenis']=='M'].groupby('cluster_masuk')['karyawan_id'].transform('count')
cluster_size_pulang = df[df['jenis']=='P'].groupby('cluster_pulang')['karyawan_id'].transform('count')

df['cluster_size_masuk']  = cluster_size_masuk
df['cluster_size_pulang'] = cluster_size_pulang
df[['cluster_size_masuk','cluster_size_pulang']] = df[['cluster_size_masuk','cluster_size_pulang']].fillna(0).astype(int)

print(f'\n✅ DBSCAN done')
print(f"Cluster MASUK  : {df['cluster_masuk'].nunique() - 1} clusters, {df['is_noise_masuk'].sum()} noise")
print(f"Cluster PULANG : {df['cluster_pulang'].nunique() - 1} clusters, {df['is_noise_pulang'].sum()} noise")

---
## 🔴 STEP 4: Estimasi Centroid Kantor per SKPD

In [ ]:
def estimate_office_location(df: pd.DataFrame) -> pd.DataFrame:
    """
    Estimasi lokasi kantor per SKPD:
    1. Ambil data MASUK saja
    2. Cari cluster terbesar per SKPD
    3. Hitung centroid (rata-rata lat/long) dari cluster terbesar
    4. Simpan sebagai office_lat, office_long
    """
    office_locations = {}
    
    masuk_df = df[df['jenis'] == 'M'].copy()
    
    for skpd_id, group in masuk_df.groupby('id_skpd'):
        # Filter bukan noise
        non_noise = group[group['cluster_masuk'] != -1]
        
        if len(non_noise) == 0:
            # Fallback: gunakan median semua titik
            office_locations[skpd_id] = {
                'office_lat':  group['lat'].median(),
                'office_long': group['long'].median()
            }
            continue
        
        # Cari cluster terbesar
        cluster_counts = non_noise['cluster_masuk'].value_counts()
        largest_cluster = cluster_counts.idxmax()
        
        # Centroid cluster terbesar
        cluster_points = non_noise[non_noise['cluster_masuk'] == largest_cluster]
        office_locations[skpd_id] = {
            'office_lat':  cluster_points['lat'].mean(),
            'office_long': cluster_points['long'].mean()
        }
    
    # Mapping ke dataframe
    # Gunakan from_dict orient='index' agar tidak ada masalah tipe data
    office_df = pd.DataFrame.from_dict(office_locations, orient='index').reset_index()
    office_df.columns = ['id_skpd', 'office_lat', 'office_long']
    
    # Samakan tipe id_skpd agar merge tidak gagal
    office_df['id_skpd'] = office_df['id_skpd'].astype(df['id_skpd'].dtype)
    
    df = df.merge(office_df, on='id_skpd', how='left')
    
    print(f'✅ Office locations estimated for {len(office_locations)} SKPDs')
    print(office_df.head())
    return df

df = estimate_office_location(df)
df[['id_skpd','office_lat','office_long']].drop_duplicates().head()

---
## 🔴 STEP 5: Haversine Distance Calculation

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Hitung jarak Haversine antara dua titik koordinat.
    Return: jarak dalam kilometer
    """
    R = 6371.0  # Radius bumi km
    
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c


def calculate_distances(df: pd.DataFrame) -> pd.DataFrame:
    """
    Hitung jarak setiap absensi ke kantor SKPD-nya.
    Tambahkan flag:
    - outside_300m : jarak > 300m
    - very_far     : jarak > 5km
    - extreme_far  : jarak > 50km (indikasi fake GPS)
    """
    df = df.copy()
    
    df['dist_km'] = haversine_distance(
        df['lat'].values,
        df['long'].values,
        df['office_lat'].values,
        df['office_long'].values
    )
    
    df['outside_300m'] = (df['dist_km'] > 0.3).astype(int)
    df['very_far']     = (df['dist_km'] > 5.0).astype(int)
    df['extreme_far']  = (df['dist_km'] > 50.0).astype(int)
    
    print('✅ Distance features created: dist_km, outside_300m, very_far, extreme_far')
    print(f"   Outside 300m : {df['outside_300m'].sum()} ({df['outside_300m'].mean()*100:.1f}%)")
    print(f"   Very far >5km: {df['very_far'].sum()} ({df['very_far'].mean()*100:.1f}%)")
    print(f"   Extreme >50km: {df['extreme_far'].sum()} ({df['extreme_far'].mean()*100:.1f}%)")
    return df

df = calculate_distances(df)
df[['karyawan_id','jenis','dist_km','outside_300m','very_far','extreme_far']].head(10)

---
## 🟤 STEP 6: Validation Logic

In [ ]:
def create_validation_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Buat feature validasi:
    - no_note         : tidak ada catatan/alasan
    - far_no_note     : jauh + tidak ada catatan (fraud kuat)
    - far_with_note   : jauh tapi ada catatan (tugas luar valid)
    - near_but_status0: dekat tapi sistem lama bilang di luar (mismatch)
    """
    df = df.copy()
    
    # Cek catatan
    df['no_note'] = df['catatan'].isna().astype(int)
    
    # Kombinasi jarak + catatan
    df['far_no_note']   = ((df['outside_300m'] == 1) & (df['no_note'] == 1)).astype(int)
    df['far_with_note'] = ((df['outside_300m'] == 1) & (df['no_note'] == 0)).astype(int)
    
    # Mismatch: dekat secara GPS tapi sistem lama bilang di luar
    df['near_but_status0'] = (
        (df['outside_300m'] == 0) & 
        (df['status_lokasi'] == 0)
    ).astype(int)
    
    # Mismatch sebaliknya: jauh tapi sistem lama bilang di dalam
    df['far_but_status1'] = (
        (df['outside_300m'] == 1) & 
        (df['status_lokasi'] == 1)
    ).astype(int)
    
    print('✅ Validation features created: no_note, far_no_note, far_with_note, near_but_status0, far_but_status1')
    print(f"   far_no_note     : {df['far_no_note'].sum()}")
    print(f"   far_with_note   : {df['far_with_note'].sum()}")
    print(f"   near_but_status0: {df['near_but_status0'].sum()}")
    print(f"   far_but_status1 : {df['far_but_status1'].sum()}")
    return df

df = create_validation_features(df)
df[['karyawan_id','dist_km','no_note','far_no_note','far_with_note','near_but_status0']].head(10)

---
## 🟣 STEP 7: ST-DBSCAN (Spatio-Temporal)

In [ ]:
def run_st_dbscan(
    df: pd.DataFrame,
    jenis: str,
    eps_km: float = 0.1,
    eps_time_hours: float = 1.0,
    min_samples: int = 3
) -> pd.Series:
    """
    ST-DBSCAN: DBSCAN dengan dimensi spasial + temporal.
    
    Menggunakan normalisasi:
    - Koordinat: lat/long dalam derajat
    - Waktu: jam_desimal (0-24)
    
    Jarak gabungan menggunakan Euclidean setelah normalisasi.
    
    Parameters:
    -----------
    eps_km         : radius spasial dalam km
    eps_time_hours : radius temporal dalam jam
    min_samples    : minimum titik
    """
    from sklearn.preprocessing import StandardScaler
    
    result = pd.Series(-1, index=df.index, dtype=int)
    subset = df[df['jenis'] == jenis].copy()
    
    # Konversi eps ke unit yang sama
    # 1 derajat lat ≈ 111 km
    eps_deg = eps_km / 111.0
    
    for skpd_id, group in subset.groupby('id_skpd'):
        if len(group) < min_samples:
            continue
        
        # Feature: lat, long, jam_desimal
        features = group[['lat', 'long', 'jam_desimal']].copy()
        
        # Normalisasi: skala waktu agar sebanding dengan spasial
        # eps_time_hours jam = eps_deg derajat (secara proporsional)
        time_scale = eps_deg / eps_time_hours
        features['jam_desimal_scaled'] = features['jam_desimal'] * time_scale
        
        X = features[['lat', 'long', 'jam_desimal_scaled']].values
        
        db = DBSCAN(
            eps=eps_deg,
            min_samples=min_samples,
            algorithm='auto',
            metric='euclidean'
        ).fit(X)
        
        labels = db.labels_
        max_label = result.max() + 1
        labels_offset = np.where(labels >= 0, labels + max_label, -1)
        result.loc[group.index] = labels_offset
    
    return result


# ============================================================
# Konfigurasi ST-DBSCAN
# ============================================================
ST_EPS_KM    = 0.1   # 100 meter
ST_EPS_HOURS = 1.0   # 1 jam
ST_MIN_SAMP  = 3

print('🔄 Running ST-DBSCAN for MASUK...')
df['st_cluster_masuk'] = run_st_dbscan(df, 'M', ST_EPS_KM, ST_EPS_HOURS, ST_MIN_SAMP)

print('🔄 Running ST-DBSCAN for PULANG...')
df['st_cluster_pulang'] = run_st_dbscan(df, 'P', ST_EPS_KM, ST_EPS_HOURS, ST_MIN_SAMP)

df['is_st_noise_masuk']  = (df['st_cluster_masuk']  == -1).astype(int)
df['is_st_noise_pulang'] = (df['st_cluster_pulang'] == -1).astype(int)

print(f'\n✅ ST-DBSCAN done')
print(f"ST Noise MASUK : {df['is_st_noise_masuk'].sum()}")
print(f"ST Noise PULANG: {df['is_st_noise_pulang'].sum()}")

---
## 🟢 STEP 8: Anomaly Scoring & Risk Level

In [ ]:
def calculate_anomaly_score(df: pd.DataFrame) -> pd.DataFrame:
    """
    Hitung anomaly_score berdasarkan kombinasi sinyal:
    
    Sinyal & Bobot:
    +3 : extreme_far (>50km, fake GPS kuat)
    +2 : very_far (>5km)
    +2 : far_no_note (jauh + tidak ada alasan)
    +1 : outside_300m (di luar 300m)
    +1 : is_noise_masuk (tidak masuk cluster manapun)
    +1 : is_st_noise_masuk (tidak konsisten waktu+lokasi)
    +1 : far_but_status1 (mismatch sistem lama)
    -1 : far_with_note (ada alasan, kurangi skor)
    
    Risk Level:
    - LOW  : score 0-1
    - MED  : score 2-3
    - HIGH : score >= 4
    """
    df = df.copy()
    
    score = pd.Series(0, index=df.index)
    
    # Bobot sinyal
    score += df.get('extreme_far',      pd.Series(0, index=df.index)) * 3
    score += df.get('very_far',         pd.Series(0, index=df.index)) * 2
    score += df.get('far_no_note',      pd.Series(0, index=df.index)) * 2
    score += df.get('outside_300m',     pd.Series(0, index=df.index)) * 1
    score += df.get('is_noise_masuk',   pd.Series(0, index=df.index)) * 1
    score += df.get('is_st_noise_masuk',pd.Series(0, index=df.index)) * 1
    score += df.get('far_but_status1',  pd.Series(0, index=df.index)) * 1
    score -= df.get('far_with_note',    pd.Series(0, index=df.index)) * 1
    
    # Pastikan tidak negatif
    score = score.clip(lower=0)
    
    df['anomaly_score'] = score.astype(int)
    
    # Risk Level
    def score_to_risk(s):
        if s >= 4:
            return 'HIGH'
        elif s >= 2:
            return 'MED'
        else:
            return 'LOW'
    
    df['risk_level'] = df['anomaly_score'].apply(score_to_risk)
    
    print('✅ Anomaly scoring done')
    print('\nDistribusi Risk Level:')
    print(df['risk_level'].value_counts())
    print('\nDistribusi Anomaly Score:')
    print(df['anomaly_score'].value_counts().sort_index())
    
    return df

df = calculate_anomaly_score(df)
df[['karyawan_id','jenis','dist_km','anomaly_score','risk_level']].head(10)

---
## 💾 STEP 9: Export Hasil

In [ ]:
# Export ke CSV untuk dipakai di Streamlit
OUTPUT_PATH = 'absensi_processed.csv'
df.to_csv(OUTPUT_PATH, index=False)
print(f'✅ Exported to {OUTPUT_PATH}')
print(f'   Shape: {df.shape}')
print(f'   Columns: {df.columns.tolist()}')

# Summary statistik
print('\n=== SUMMARY ===')
print(f"Total absensi    : {len(df)}")
print(f"Total karyawan   : {df['karyawan_id'].nunique()}")
print(f"Total SKPD       : {df['id_skpd'].nunique()}")
print(f"HIGH risk        : {(df['risk_level']=='HIGH').sum()} ({(df['risk_level']=='HIGH').mean()*100:.1f}%)")
print(f"MED risk         : {(df['risk_level']=='MED').sum()} ({(df['risk_level']=='MED').mean()*100:.1f}%)")
print(f"LOW risk         : {(df['risk_level']=='LOW').sum()} ({(df['risk_level']=='LOW').mean()*100:.1f}%)")

---
## 🗺️ Quick Visualization (Folium) - Preview di Colab

In [ ]:
import folium
from folium.plugins import HeatMap, MarkerCluster

def create_preview_map(df: pd.DataFrame, max_points: int = 500) -> folium.Map:
    """
    Buat peta preview dengan Folium.
    Warna marker berdasarkan risk_level:
    - HIGH : merah
    - MED  : oranye
    - LOW  : hijau
    """
    color_map = {'HIGH': 'red', 'MED': 'orange', 'LOW': 'green'}
    
    # Center peta
    center_lat  = df['lat'].median()
    center_long = df['long'].median()
    
    m = folium.Map(
        location=[center_lat, center_long],
        zoom_start=12,
        tiles='OpenStreetMap'
    )
    
    # Sample data agar tidak terlalu berat
    sample = df.sample(min(max_points, len(df)), random_state=42)
    
    # Marker cluster
    mc = MarkerCluster().add_to(m)
    
    for _, row in sample.iterrows():
        color = color_map.get(row.get('risk_level', 'LOW'), 'blue')
        popup_text = f"""
        <b>Karyawan:</b> {row['karyawan_id']}<br>
        <b>SKPD:</b> {row['id_skpd']}<br>
        <b>Jenis:</b> {row['jenis']}<br>
        <b>Waktu:</b> {row['tanggal_kirim']}<br>
        <b>Jarak ke kantor:</b> {row.get('dist_km', 0):.3f} km<br>
        <b>Risk Level:</b> <span style='color:{color}'><b>{row.get('risk_level','N/A')}</b></span><br>
        <b>Anomaly Score:</b> {row.get('anomaly_score', 0)}
        """
        folium.CircleMarker(
            location=[row['lat'], row['long']],
            radius=6,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(popup_text, max_width=250)
        ).add_to(mc)
    
    # Tambahkan marker kantor
    offices = df[['id_skpd','office_lat','office_long']].drop_duplicates()
    for _, row in offices.iterrows():
        if pd.notna(row['office_lat']):
            folium.Marker(
                location=[row['office_lat'], row['office_long']],
                popup=f"Kantor SKPD {row['id_skpd']}",
                icon=folium.Icon(color='blue', icon='building', prefix='fa')
            ).add_to(m)
    
    return m

preview_map = create_preview_map(df)
preview_map  # Tampilkan di Colab

### 🔥 Heatmap Anomali

In [ ]:
# Heatmap khusus HIGH risk
high_risk = df[df['risk_level'] == 'HIGH']

center_lat  = df['lat'].median()
center_long = df['long'].median()

heatmap = folium.Map(
    location=[center_lat, center_long],
    zoom_start=12
)

heat_data = high_risk[['lat','long']].dropna().values.tolist()
HeatMap(heat_data, radius=15, blur=10, max_zoom=13).add_to(heatmap)

print(f'Heatmap dari {len(heat_data)} titik HIGH risk')
heatmap